In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import math
import scipy.stats as st

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
def score_up_down_diff(up, down):
    return up - down

In [4]:
score_up_down_diff(600, 400)

200

In [5]:
score_up_down_diff(5500, 4500)

1000

In [6]:
def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)

In [7]:
score_average_rating(2, 0)

1.0

In [8]:
score_average_rating(100, 1)

0.9900990099009901

In [9]:
def wilson_lower_bound(up, down, confidence=0.95):
    """
    Wilson Lower Bound Score Calculation
    - WLB Score is admitted as confidence interval lower bound of calculating Bernoulli parameter p
    - This calculated score is used for sorting.
    - Note:
    If scores are between 1 to 5, then scores 1,2,3 are negative and scores of 4, 5 are positive. It fits for bernoulli.
    In this case, Some issues occur. It should be done bayesian average rating.

    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float

    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [10]:
wilson_lower_bound(600, 400)

0.5693094295142663

In [11]:
wilson_lower_bound(5500, 4500)

0.5402319557715324

In [12]:
wilson_lower_bound(2, 0)

0.3423802275066531

In [13]:
wilson_lower_bound(100, 1)

0.9460328420055449

In [14]:
up = [15, 70, 14, 4, 2, 5, 8, 37, 21, 52, 28, 147, 61, 30, 23, 40, 37, 61, 54, 18, 12, 68]
down = [0, 2, 2, 2, 15, 2, 6, 5, 23, 8, 12, 2, 1, 1, 5, 1, 2, 6, 2, 0, 2, 2]
comments = pd.DataFrame({"up": up, "down": down})

In [15]:
comments["score_pos_neg_diff"] = comments.apply(lambda x: score_up_down_diff(x["up"],
                                                                             x["down"]), axis=1)

In [16]:
comments["score_average_rating"] = comments.apply(lambda x: score_average_rating(x["up"], x["down"]), axis=1)

In [17]:
comments["wilson_lower_bound"] = comments.apply(lambda x: wilson_lower_bound(x["up"], x["down"]), axis=1)

In [18]:
comments.sort_values("wilson_lower_bound", ascending=False)

,up,down,score_pos_neg_diff,score_average_rating,wilson_lower_bound
11,147,2,145,0.98658,0.95238
12,61,1,60,0.98387,0.91413
1,70,2,68,0.97222,0.90426
21,68,2,66,0.97143,0.90168
18,54,2,52,0.96429,0.87881
15,40,1,39,0.97561,0.87405
13,30,1,29,0.96774,0.83806
16,37,2,35,0.94872,0.83114
19,18,0,18,1.00000,0.82412
17,61,6,55,0.91045,0.81807
